# sveltejs/sveltekit

> 

In [1]:
#| default_exp sveltejs-sveltekit

In [2]:
#| hide
import nbdev; nbdev.nbdev_export()

In [3]:
#| export
from lovely_docs.settings import settings, GitSource
from lovely_docs.git import clone_repo
from lovely_docs.docs import (
    build_markdown_doc_tree,
    process_tree_depth_first,
    calculate_total_usage,
    calculate_cost,
    save_processed_documents
)
import llm
from llm.models import Usage
import anthropic
from pydantic import BaseModel, Field
from jinja2 import Environment, FileSystemLoader
import asyncio
import json
import shutil
from pathlib import Path
import langsmith as ls
import logging

In [4]:
#| export

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

#### Clone the repo

In [5]:
#| export

source = settings.sources["sveltejs/sveltekit"]
assert isinstance(source, GitSource)

commit = clone_repo(source)
source.commit = commit # Replace the literal commit (master) with the hash
clone_dir = settings.git_dir / source.name

#### Read the markdown files

In [6]:
tree = build_markdown_doc_tree(clone_dir/source.doc_dir)
# tree.model_dump()

#### Process the tree

In [7]:
# tree.subdirs = tree.subdirs[:2] # For testing process just the first 2 secstion (intro and runes)

In [8]:
processed_tree = await process_tree_depth_first(settings, tree, source.name)

+0.446s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 10-getting-started/10-introduction.md


+0.318s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 10-getting-started/25-project-types.md
+0.016s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 10-getting-started/20-creating-a-project.md
+0.016s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 10-getting-started/30-project-structure.md
+0.024s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 10-getting-started/40-web-standards.md
+0.014s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 10-getting-started/index.md
+0.014s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 20-core-concepts/10-routing.md
+0.014s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 20-core-concepts/20-load.md
+0.015s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 20-core-concepts/30-form-actions.md
+0.016s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 20-core-concepts/40-page-options.md
+0.020s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 25-build-and-dep

#### Calculate cost

In [9]:
usage =calculate_total_usage(processed_tree)
print(f"\nTotal Usage:")
print(f"  Input tokens: {usage.input:,}")
print(f"  Output tokens: {usage.output:,}")
print(f"  Total tokens: {(usage.input + usage.output):,}")

cost, input_cost, output_cost = calculate_cost(usage, 1, 5)

print(f"Cost:")
print(f"  Total: ${cost:.2f}")
print(f"  Input: ${input_cost:.2f}")
print(f"  Output: ${output_cost:.2f}")


Total Usage:
  Input tokens: 213,892
  Output tokens: 63,438
  Total tokens: 277,330
Cost:
  Total: $0.53
  Input: $0.21
  Output: $0.32


#### save the results

In [10]:
save_processed_documents(source, settings.output_dir / source.name, processed_tree)

+86.727s DEBUG lovely_docs/docs.py:367 file_map Added . ->  (.)
+0.001s DEBUG lovely_docs/docs.py:367 file_map Added getting-started -> getting-started (10-getting-started)
+0.000s DEBUG lovely_docs/docs.py:375 file_map Added page getting-started/introduction -> introduction (10-getting-started/10-introduction.md)
+0.000s DEBUG lovely_docs/docs.py:375 file_map Added page getting-started/creating-a-project -> creating-a-project (10-getting-started/20-creating-a-project.md)
+0.000s DEBUG lovely_docs/docs.py:375 file_map Added page getting-started/project-types -> project-types (10-getting-started/25-project-types.md)
+0.000s DEBUG lovely_docs/docs.py:375 file_map Added page getting-started/project-structure -> project-structure (10-getting-started/30-project-structure.md)
+0.000s DEBUG lovely_docs/docs.py:375 file_map Added page getting-started/web-standards -> web-standards (10-getting-started/40-web-standards.md)
+0.000s DEBUG lovely_docs/docs.py:375 file_map Added page getting-started

In [11]:
processed_tree.subdirs[-1].pages

[DocPage(path=Path('98-reference/10-@sveltejs-kit.md'), name='@sveltejs∕kit-api-reference', digest='This page documents the @sveltejs/kit module API. It references private types that are used internally by public types but cannot be imported directly by users. The page serves as a reference for the public API surface of the kit module.', short_digest='Reference documentation for @sveltejs/kit module. Notes that private types exist but cannot be imported directly.', essence='API reference page for @sveltejs/kit module with notes on private vs public types.', relevant=False, usage=Usage(input=1104, output=195, details=None), type='page', token_counts=TokenCounts(fulltext=58, digest=55, short_digest=29), fulltext='---\ntitle: @sveltejs/kit\n---\n\n> MODULE: @sveltejs/kit\n\n## Private types\n\nThe following are referenced by the public types documented above, but cannot be imported directly:\n\n> TYPES: Private types\n'),
 DocPage(path=Path('98-reference/15-@sveltejs-kit-hooks.md'), name=